In [1]:
!pip install ipynb sklearn progressbar2

     |████████████████████████████████| 6.8 MB 5.3 MB/s eta 0:00:01
     |████████████████████████████████| 300 kB 4.6 MB/s eta 0:00:01
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=2397 sha256=db605f8297deba7f58cc3aa7c05dea1807ea884577b9bd073244a179ed7eeeef
  Stored in directory: /root/.cache/pip/wheels/23/9d/42/5ec745cbbb17517000a53cecc49d6a865450d1f5cb16dc8a9c
Successfully built sklearn
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [2]:
from ipynb.fs.full.utils import *
import numpy as np
import matplotlib.pyplot as plt
import pickle

In [3]:
res = 24

In [4]:
def draw(imgs, labels):
    fig, axs = plt.subplots(1, len(imgs), figsize=(15, 3), sharey=True, subplot_kw=dict(projection='3d'))
    for i in range(len(imgs)):
        cube1 = (imgs[i][:,:,:] >= 1)
        axs[i].voxels(cube1, facecolors="blue")
        axs[i].set_title(labels[i])
    plt.show()

In [5]:
class Histogram:
    def __init__(self, histX, histY, histZ, label):
        self.histX = histX
        self.histY = histY
        self.histZ = histZ
        self.label = label
        
    @classmethod
    def load(self, filename):
        with open(filename + '.pkl', 'rb') as input:
            return pickle.load(input)
    
    def save(self, filename):
        with open(filename + '.pkl', 'wb') as output:
            pickle.dump(self, output, pickle.HIGHEST_PROTOCOL)
            
    def draw(self):
        fig, axs = plt.subplots(1, 3, figsize=(15, 3), sharey=True)
        axs[0].bar(range(res), self.histX)
        axs[0].set_title(self.label + ' X')
        axs[1].bar(range(res), self.histY)
        axs[1].set_title('Y')
        axs[2].bar(range(res), self.histZ)
        axs[2].set_title('Z')
        plt.show()
        
    def getData(self):
        return [self.histX, self.histY, self.histZ]

In [6]:
def calculate_histogram(obj, label, res):
    objy = np.moveaxis(obj, [0, 1, 2], [1, 0, 2])
    objz = np.moveaxis(obj, [0, 1, 2], [1, 2, 0])
    #calculate the histogram in 3 axis
    histx = np.zeros(res, dtype=int)
    histy = np.zeros(res, dtype=int)
    histz = np.zeros(res, dtype=int)
    for i in range(res):
        ox = obj[i].reshape([res * res])
        histx[i] = ox[ox == 1].size

        oy = objy[i].reshape([res * res])
        histy[i] = oy[oy == 1].size

        oz = objz[i].reshape([res * res])
        histz[i] = oz[oz == 1].size
    return Histogram(histx, histy, histz, label)


In [7]:
def draw_histogram(histograms):
    fig, axs = plt.subplots(1, 3, figsize=(15, 3), sharey=True)
    axs[0].bar(range(res), histograms.histX)
    axs[0].set_title(label + ' X')
    axs[1].bar(range(res), histograms.histY)
    axs[1].set_title('Y')
    axs[2].bar(range(res), histograms.histZ)
    axs[2].set_title('Z')
    plt.show()

In [8]:
def load_histograms(source_labels, dataset = 'train', verbosity=0, shuffre = True):
    bar = progressbar.ProgressBar(max_value=len(source_labels)-1)
    data = []
    labels = np.empty(0, dtype=int)
    for i, label in enumerate(source_labels):
        histogram = Histogram.load('histograms/' + dataset + '-' + label + '-' + str(i))
        data.append(histogram.getData())
        labels = np.append(labels, i)
    
    return (data, labels)

In [ ]:
a, b = load_histograms(all_labels, 'train', 1, False)
len(a)

In [ ]:
a = []
a.append([1,2,3])
a.append([4,5,6])
a

In [ ]:
training_histograms[0]

# Calculate and save histograms

In [8]:
bar2 = progressbar.ProgressBar(max_value=len(all_labels)-1)

for i, label in enumerate(all_labels):
    bar2.update(i)
    training, training_labels = load_data(res, [label], 'train', 1, 0, False)
    for j in range(len(training)):
        histograms = calculate_histogram(training[j], label, res)
        histograms.save('histograms/train-' + histograms.label + '-' + str(j))
    test, testlabels = load_data(res, [label], 'test', 1, False, False)
    for j in range(len(test)):
        histograms = calculate_histogram(test[j], label, res)
        histograms.save('histograms/test-' + histograms.label + '-' + str(j))


100% (39 of 39) |########################| Elapsed Time: 0:00:20 ETA:  00:00:00

In [16]:
bar2 = progressbar.ProgressBar(max_value=len(all_labels)-1)
training_histograms = []
test_histograms = []
training_histograms_labels = np.empty(0, int)
test_histograms_labels = np.empty(0, int)

for i, label in enumerate(all_labels):
    bar2.update(i)
    training, training_labels = load_data(res, [label], 'train', 1, 0, False)
    for j in range(len(training)):
        training_histograms.append(calculate_histogram(training[j], label, res).getData())
    training_histograms_labels = np.concatenate((training_histograms_labels, np.full(len(training), i, dtype=int)))

    test, testlabels = load_data(res, [label], 'test', 1, False, False)
    for j in range(len(test)):
        test_histograms.append(calculate_histogram(test[j], label, res).getData())
    test_histograms_labels = np.concatenate((test_histograms_labels, np.full(len(test), i, dtype=int)))

rng_state = np.random.get_state()
np.random.shuffle(training_histograms)
np.random.set_state(rng_state)
np.random.shuffle(training_histograms_labels)
test_histograms
test_histograms_labels


100% (39 of 39) |########################| Elapsed Time: 0:00:15 ETA:  00:00:00

array([ 0,  0,  0, ..., 39, 39, 39])

In [ ]:
training_histograms_labels

# Prepare Network 

In [10]:
class Model:
    def __init__(self, res, num_classes, verbosity):
        self.prepare_model(res, num_classes, verbosity)
        
    def prepare_model(self, res, num_classes, verbosity = 0):
        pass
    
    def train(self, training, training_labels, res, num_classes, verbosity = 0, log = False, epochs = None):
        log_dir = "logs/fit/" + str(res) + '-' + str(num_classes) + '-' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
        if verbosity == 1 and log:
            print("Log file " + log_dir)
        if log:
            tensorboard_callbacks = [tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)]
        else:
            tensorboard_callbacks = None

        validation_split = 0.1
        if epochs == None:
            epochs = 2 * num_classes

        self.model.fit(
            self.prepare_data(training, num_classes, res), 
            training_labels, 
            epochs=epochs,
            verbose=verbosity,
            callbacks=tensorboard_callbacks,
            validation_split=validation_split
    )
    
    def evaluate(self, test, test_labels, verbose):
        return self.model.evaluate(
            test,
            test_labels, 
            verbose=2
        )

In [11]:
class ModelDense(Model): 
    
    def __init__(self, res, num_classes, verbosity):
        super().__init__(res, num_classes, verbosity)

    def prepare_model(self, res, num_classes, verbosity = 0):
        self.model = keras.Sequential([
            keras.layers.Flatten(input_shape=(res, res, res)),
            keras.layers.Dense(res*2, activation='relu'),
            keras.layers.Dense(res/2, activation='sigmoid'),
            keras.layers.Dropout(.2, input_shape=(2,)),
            keras.layers.Dense(40, activation='softmax')
        ])

        learning_rate = 0.0001

        self.model.compile(optimizer=keras.optimizers.Adam(lr=learning_rate),
                      loss='sparse_categorical_crossentropy',
                      metrics=['accuracy'])
        if (verbosity == 1):
            print(self.model.summary())
            
    def prepare_data(self, data, num_classes, res):
        return data
    model_type = 'dense'

In [32]:
class ModelConv3D(Model):

    def __init__(self, res, num_classes, verbosity):
        super().__init__(res, num_classes, verbosity)
    
    def prepare_model(self, res, num_classes, verbosity = 0):
        self.model = keras.Sequential([
            keras.layers.Conv2D(
                    num_classes * 3, 
                    kernel_size=(2),
#                    strides=(2),
                    activation='relu', 
                    kernel_initializer='he_uniform', 
                    data_format="channels_last",
                    input_shape=(3, res, 1)
                ),        
            #keras.layers.Conv3D(
            #        res, 
            #        kernel_size=(5),
            #        strides=(2),
            #        activation='relu', 
            #        kernel_initializer='he_uniform'
            #    ),        
            #keras.layers.Conv3D(
            #        res, 
            #        kernel_size=(2),
            #        activation='relu', 
            #        kernel_initializer='he_uniform'
            #    ),        

            #keras.layers.MaxPooling2D(pool_size=(2, 2), strides=None, padding="valid", data_format=None),
            keras.layers.Flatten(),
            #keras.layers.Dense(res/2, activation='relu'),
            keras.layers.Dense(num_classes, activation='softmax')
        ])

        learning_rate = 0.0001

        self.model.compile(optimizer=keras.optimizers.Adam(lr=learning_rate),
                      loss='sparse_categorical_crossentropy',
                      metrics=['accuracy'])
        if (verbosity == 1):
            print(self.model.summary())

    def prepare_data(self, data, num_classes, res):
        return np.array(data).reshape(len(data), 3, res, 1)
    model_type = 'conv3d'


In [33]:
num_classes = len(all_labels)
model = ModelConv3D(res, num_classes, 1)
model.train(training_histograms, training_histograms_labels, res, num_classes, 2, False, 500)


Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 2, 23, 120)        600       
_________________________________________________________________
flatten_5 (Flatten)          (None, 5520)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 40)                220840    
Total params: 221,440
Trainable params: 221,440
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/500
276/276 - 1s - loss: 27.9868 - accuracy: 0.3674 - val_loss: 12.9351 - val_accuracy: 0.5025
Epoch 2/500
276/276 - 1s - loss: 10.1092 - accuracy: 0.5599 - val_loss: 10.0482 - val_accuracy: 0.5627
Epoch 3/500
276/276 - 1s - loss: 7.3525 - accuracy: 0.6097 - val_loss: 9.2634 - val_accuracy: 0.5627
Epoch 4/500
276/276 - 1s - loss: 6.7284 - accuracy: 0.6281 - val_l

Epoch 74/500
276/276 - 1s - loss: 1.3689 - accuracy: 0.8698 - val_loss: 13.5335 - val_accuracy: 0.7115
Epoch 75/500
276/276 - 1s - loss: 1.4774 - accuracy: 0.8708 - val_loss: 12.5617 - val_accuracy: 0.6993
Epoch 76/500
276/276 - 1s - loss: 1.3627 - accuracy: 0.8766 - val_loss: 11.6792 - val_accuracy: 0.6962
Epoch 77/500
276/276 - 1s - loss: 1.5510 - accuracy: 0.8674 - val_loss: 11.6808 - val_accuracy: 0.7227
Epoch 78/500
276/276 - 1s - loss: 1.5325 - accuracy: 0.8644 - val_loss: 11.4191 - val_accuracy: 0.7401
Epoch 79/500
276/276 - 1s - loss: 1.2578 - accuracy: 0.8718 - val_loss: 11.0866 - val_accuracy: 0.7187
Epoch 80/500
276/276 - 1s - loss: 1.4557 - accuracy: 0.8761 - val_loss: 11.7888 - val_accuracy: 0.7085
Epoch 81/500
276/276 - 1s - loss: 1.1456 - accuracy: 0.8828 - val_loss: 11.7104 - val_accuracy: 0.7227
Epoch 82/500
276/276 - 1s - loss: 1.2754 - accuracy: 0.8747 - val_loss: 11.3226 - val_accuracy: 0.7176
Epoch 83/500
276/276 - 2s - loss: 1.3412 - accuracy: 0.8764 - val_loss: 1

276/276 - 2s - loss: 0.6328 - accuracy: 0.9258 - val_loss: 14.9493 - val_accuracy: 0.7217
Epoch 154/500
276/276 - 2s - loss: 0.6959 - accuracy: 0.9242 - val_loss: 15.5776 - val_accuracy: 0.7238
Epoch 155/500
276/276 - 2s - loss: 0.6782 - accuracy: 0.9301 - val_loss: 13.9845 - val_accuracy: 0.7339
Epoch 156/500
276/276 - 2s - loss: 0.5701 - accuracy: 0.9294 - val_loss: 14.1320 - val_accuracy: 0.7411
Epoch 157/500
276/276 - 3s - loss: 0.5390 - accuracy: 0.9327 - val_loss: 15.5789 - val_accuracy: 0.7115
Epoch 158/500
276/276 - 3s - loss: 0.6344 - accuracy: 0.9227 - val_loss: 14.0675 - val_accuracy: 0.7288
Epoch 159/500
276/276 - 2s - loss: 0.7687 - accuracy: 0.9182 - val_loss: 14.4045 - val_accuracy: 0.7146
Epoch 160/500
276/276 - 2s - loss: 0.7833 - accuracy: 0.9204 - val_loss: 14.4196 - val_accuracy: 0.7166
Epoch 161/500
276/276 - 2s - loss: 0.6651 - accuracy: 0.9229 - val_loss: 14.6694 - val_accuracy: 0.7258
Epoch 162/500
276/276 - 2s - loss: 0.7813 - accuracy: 0.9178 - val_loss: 15.34

Epoch 232/500
276/276 - 1s - loss: 0.4533 - accuracy: 0.9455 - val_loss: 16.7006 - val_accuracy: 0.7278
Epoch 233/500
276/276 - 1s - loss: 0.3405 - accuracy: 0.9555 - val_loss: 15.4869 - val_accuracy: 0.7217
Epoch 234/500
276/276 - 2s - loss: 0.4135 - accuracy: 0.9494 - val_loss: 16.4708 - val_accuracy: 0.7299
Epoch 235/500
276/276 - 2s - loss: 0.3785 - accuracy: 0.9508 - val_loss: 16.1568 - val_accuracy: 0.7523
Epoch 236/500
276/276 - 1s - loss: 0.4510 - accuracy: 0.9472 - val_loss: 17.3782 - val_accuracy: 0.7492
Epoch 237/500
276/276 - 1s - loss: 0.4961 - accuracy: 0.9459 - val_loss: 15.8986 - val_accuracy: 0.7533
Epoch 238/500
276/276 - 1s - loss: 0.3867 - accuracy: 0.9463 - val_loss: 17.1044 - val_accuracy: 0.7370
Epoch 239/500
276/276 - 1s - loss: 0.3659 - accuracy: 0.9494 - val_loss: 16.6315 - val_accuracy: 0.7390
Epoch 240/500
276/276 - 1s - loss: 0.5103 - accuracy: 0.9397 - val_loss: 16.7376 - val_accuracy: 0.7472
Epoch 241/500
276/276 - 1s - loss: 0.3651 - accuracy: 0.9527 - v

Epoch 311/500
276/276 - 1s - loss: 0.2649 - accuracy: 0.9598 - val_loss: 16.5776 - val_accuracy: 0.7452
Epoch 312/500
276/276 - 1s - loss: 0.3548 - accuracy: 0.9603 - val_loss: 17.1427 - val_accuracy: 0.7482
Epoch 313/500
276/276 - 1s - loss: 0.3374 - accuracy: 0.9558 - val_loss: 17.3716 - val_accuracy: 0.7339
Epoch 314/500
276/276 - 1s - loss: 0.3417 - accuracy: 0.9595 - val_loss: 16.9366 - val_accuracy: 0.7462
Epoch 315/500
276/276 - 1s - loss: 0.2951 - accuracy: 0.9594 - val_loss: 17.3343 - val_accuracy: 0.7431
Epoch 316/500
276/276 - 2s - loss: 0.3046 - accuracy: 0.9603 - val_loss: 17.5535 - val_accuracy: 0.7421
Epoch 317/500
276/276 - 1s - loss: 0.3045 - accuracy: 0.9609 - val_loss: 17.9484 - val_accuracy: 0.7421
Epoch 318/500
276/276 - 2s - loss: 0.2834 - accuracy: 0.9623 - val_loss: 17.2734 - val_accuracy: 0.7411
Epoch 319/500
276/276 - 1s - loss: 0.2908 - accuracy: 0.9655 - val_loss: 16.5072 - val_accuracy: 0.7523
Epoch 320/500
276/276 - 1s - loss: 0.2996 - accuracy: 0.9635 - v

Epoch 390/500
276/276 - 1s - loss: 0.2417 - accuracy: 0.9663 - val_loss: 18.1623 - val_accuracy: 0.7523
Epoch 391/500
276/276 - 1s - loss: 0.2554 - accuracy: 0.9670 - val_loss: 18.7995 - val_accuracy: 0.7503
Epoch 392/500
276/276 - 1s - loss: 0.2918 - accuracy: 0.9635 - val_loss: 17.9709 - val_accuracy: 0.7452
Epoch 393/500
276/276 - 1s - loss: 0.2660 - accuracy: 0.9658 - val_loss: 18.0044 - val_accuracy: 0.7401
Epoch 394/500
276/276 - 1s - loss: 0.2228 - accuracy: 0.9658 - val_loss: 17.9912 - val_accuracy: 0.7309
Epoch 395/500
276/276 - 1s - loss: 0.2217 - accuracy: 0.9695 - val_loss: 18.3161 - val_accuracy: 0.7543
Epoch 396/500
276/276 - 1s - loss: 0.2132 - accuracy: 0.9704 - val_loss: 17.4191 - val_accuracy: 0.7452
Epoch 397/500
276/276 - 1s - loss: 0.3346 - accuracy: 0.9644 - val_loss: 19.5001 - val_accuracy: 0.7370
Epoch 398/500
276/276 - 1s - loss: 0.2592 - accuracy: 0.9640 - val_loss: 18.6606 - val_accuracy: 0.7401
Epoch 399/500
276/276 - 1s - loss: 0.2990 - accuracy: 0.9650 - v

Epoch 469/500
276/276 - 1s - loss: 0.2066 - accuracy: 0.9714 - val_loss: 17.6523 - val_accuracy: 0.7533
Epoch 470/500
276/276 - 1s - loss: 0.2133 - accuracy: 0.9679 - val_loss: 18.9405 - val_accuracy: 0.7278
Epoch 471/500
276/276 - 1s - loss: 0.2427 - accuracy: 0.9687 - val_loss: 18.6635 - val_accuracy: 0.7452
Epoch 472/500
276/276 - 1s - loss: 0.2811 - accuracy: 0.9676 - val_loss: 18.2318 - val_accuracy: 0.7350
Epoch 473/500
276/276 - 1s - loss: 0.2276 - accuracy: 0.9704 - val_loss: 17.9129 - val_accuracy: 0.7564
Epoch 474/500
276/276 - 1s - loss: 0.2446 - accuracy: 0.9696 - val_loss: 18.3526 - val_accuracy: 0.7574
Epoch 475/500
276/276 - 1s - loss: 0.2126 - accuracy: 0.9702 - val_loss: 18.7513 - val_accuracy: 0.7554
Epoch 476/500
276/276 - 1s - loss: 0.2374 - accuracy: 0.9721 - val_loss: 18.3900 - val_accuracy: 0.7472
Epoch 477/500
276/276 - 1s - loss: 0.1546 - accuracy: 0.9752 - val_loss: 18.3754 - val_accuracy: 0.7472
Epoch 478/500
276/276 - 1s - loss: 0.2774 - accuracy: 0.9696 - v

In [34]:
test_loss, test_acc = model.evaluate(
    model.prepare_data(test_histograms, num_classes, res),  
    test_histograms_labels, 
    verbose=2
)


76/76 - 0s - loss: 21.1137 - accuracy: 0.7414
